In [ ]:
import os
import operator
import pandas as pd
import seaborn as sns
from functools import reduce

In [ ]:
# Parms
exam_in = ''

In [ ]:
path = '/media/marcos/Dados/Projects/Results/PupilDeep/Frames'

In [ ]:
def make_path(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        pass

In [ ]:
path_in = '{}/{}'.format(path, exam_in)
path_out = '{}/{}/{}'.format(path, exam_in, 'graphs_analysis')
print(path_in)
print(path_out)

analysis_list = [x for x in os.listdir(path_in) if ('.txt' in x) and ('analysis' in x)]
print(analysis_list)

if len(analysis_list) > 1:
    raise Exception('Found more one files for analysis!')
else:
    path_file = '{}/{}'.format(path_in, analysis_list[0])
    print(path_file)

make_path(path_out)

In [ ]:
def read_benchmark_analysis(path):
    frames = []
    fails = []
    with open(path) as benchmark_file:
        for line in benchmark_file.readlines():
            line = line.rstrip('\r\n')
            if ('[' in line) and (']' in line):
                frames.append(line)
            elif 'Falha' in line:
                fails.append(line)
            elif line == '':
                continue
            elif '***' in line:
                break            
    
    if len(frames) == len(fails):
        return frames, fails
    else:
        raise 'Fail! Exists more frames than fails!'

In [ ]:
def agrupe_fails(frames, fails):
    intervals = []
    number_fail = -1
    for frame in frames:
        number_fail += 1
        while '[' in frame:
            interval = frame[frame.index('[') : frame.index(']') + 1]
            frame = frame[frame.index(']')+1: len(frame)]
            intervals.append({'interval': interval, 'fail': fails[number_fail]})

    new_intervals = []
    for interval in intervals:
        new_interval = []

        str_interval = interval['interval']
        str_interval = str_interval.replace('[', '').replace(']','')
        if 'a' in str_interval:
            init_range = int(str_interval[0 : str_interval.index('a')-1])
            final_range = int(str_interval[str_interval.index('a')+2 : len(str_interval)])
            new_interval = [n for n in range(init_range, final_range, 1)]
        else:
            new_interval.append(int(str_interval))

        new_intervals.append({'interval': new_interval, 'fail': interval['fail']})

    new_fails = []
    for i in range(len(fails)):
        str_fail = fails[i]
        str_fail = str_fail[str_fail.index(':')+2 : len(str_fail)-1]
        if str_fail not in fails:
            new_fails.append(str_fail)

    fails_intervals = []
    for fail in fails:
        fail_range = [x['interval'] for x in new_intervals if fail in x['fail']]
        fail_range_unique = reduce(operator.concat, fail_range)
        fails_intervals.append({'fail': fail,
                                'number_fails': len(fail_range_unique),
                                'intervals': fail_range_unique})

    return fails_intervals

In [ ]:
def create_csv_out(name_csv, fails_intervals_out):
    path_writer = '{}/{}.csv'.format(path_out, name_csv)
    print(path_writer)

    with open(path_writer, 'a') as file_out:
        file_out.write('fail,frame,index_fail\n')

        for fail in fails_intervals_out:
            number_of_file = 1
            for frame in fail['intervals']:
                str_out = '"{}",{},{}\n'.format(fail['fail'], frame, number_of_file)
                file_out.write(str_out)
                number_of_file += 1

In [ ]:
def create_csv_resume(name_csv_resume, fails_intervals_out):
    path_writer = '{}/{}.csv'.format(path_out, name_csv_resume)

    with open(path_writer, 'a') as file_out:
        file_out.write('fail,quantity\n')
        
        for fail in fails_intervals_out:
            str_fail = fail['fail'].split(' ')
            str_fail = list(map(lambda x: x.title(), str_fail))
            str_fail = ''.join(str_fail[2:4])
            
            str_out = '"{}",{}\n'.format(str_fail, len(fail['intervals']))
            file_out.write(str_out)
    try:
        os.mkdir(path)
    except FileExistsError:
        pass

In [ ]:
def plot_boxplot(dataset, title):
    img = sns.barplot(x='fail', y='quantity', data=dataset, palette='hls', capsize=0.05,
                      saturation=8, errcolor='gray', errwidth=2, ci='sd')
    img.figure.savefig('{}/{}.pdf'.format(path_out, title))

In [ ]:
frames, fails = read_benchmark_analysis(path_file)
print(frames[0:3], fails[0:3])

In [ ]:
dataset_benchmark = agrupe_fails(frames, fails)
print(dataset_benchmark[0])

In [ ]:
name_data_benchmark = 'dataset_benchmark'
create_csv_out(name_data_benchmark, dataset_benchmark)

name_data_resume_benchmark = 'dataset_benchmark_resume'
create_csv_resume(name_data_resume_benchmark, dataset_benchmark)

In [ ]:
path_data = '{}/{}.csv'.format(path_out, name_data_resume_benchmark)
data_benchmark = pd.read_csv(path_data)
print(data_benchmark.head())

In [ ]:
plot_boxplot(data_benchmark, 'resume_benchmark')